In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, normalize, RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [5]:
train_df = pd.read_csv("../Data/Train.csv")

In [6]:
X = train_df.iloc[:,1:]
y = X.pop("CLASS")

In [7]:
X.head()

,GENERAL APPEARANCE,MANNER OF SPEAKING,PHYSICAL CONDITION,MENTAL ALERTNESS,SELF-CONFIDENCE,ABILITY TO PRESENT IDEAS,COMMUNICATION SKILLS,Student Performance Rating,TOTAL_SCORE,AVG_SCORE
0,4,4,4,4,4,4,4,5,33,4.125
1,5,5,5,5,5,4,5,5,39,4.875
2,4,5,5,3,3,5,3,5,33,4.125
3,4,5,4,5,4,4,4,5,35,4.375
4,3,3,3,3,3,3,3,5,26,3.250


In [8]:
scaler= '../tools/power_transform.sav'
scaler = pickle.load(open(scaler, 'rb'))
X_train_scaled = scaler.transform(X)

C:\Users\ML_USER\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ML_USER\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator PowerTransformer from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
kfold = StratifiedKFold(random_state=42, n_splits=10, shuffle=True)
rfc =  RandomForestClassifier(n_estimators=250)
acc_hist = []

for train_idx, val_idx in kfold.split(X_train_scaled, y):
    X_train, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict_proba(X_val)[:,1]
    classif = classification_report(y_val, np.round(y_pred), output_dict=True)
    classif_pd = pd.DataFrame(classif).T
    acc_hist.append(classif["accuracy"])
    print(classif_pd)
    print("\n")


print(f"Mean Accuracy: {np.mean(acc_hist)}")

    

              precision    recall  f1-score     support
0              0.861111  0.920792  0.889952  101.000000
1              0.938931  0.891304  0.914498  138.000000
accuracy       0.903766  0.903766  0.903766    0.903766
macro avg      0.900021  0.906048  0.902225  239.000000
weighted avg   0.906045  0.903766  0.904125  239.000000


              precision    recall  f1-score     support
0              0.896907  0.861386  0.878788  101.000000
1              0.901408  0.927536  0.914286  138.000000
accuracy       0.899582  0.899582  0.899582    0.899582
macro avg      0.899158  0.894461  0.896537  239.000000
weighted avg   0.899506  0.899582  0.899285  239.000000


              precision    recall  f1-score    support
0              0.906250  0.870000  0.887755  100.00000
1              0.909091  0.935252  0.921986  139.00000
accuracy       0.907950  0.907950  0.907950    0.90795
macro avg      0.907670  0.902626  0.904870  239.00000
weighted avg   0.907902  0.907950  0.907663  239.

In [18]:
file_location = '../Models/RFC_model.sav'
pickle.dump(rfc, open(file_location, 'wb'))